# <h1><b><i><center><font color="orange"><u>Algoritmo Genético</u></font></center></i></b></h1>
<br/>

# School of AI - São Paulo
## Professor:   Victor Venites
## Data: 03/12/2019
## Aula 23

# Algoritmo Genetico

In [1]:
%%html
<marquee style='width: 100%; color: blue;'><h1><b><i>Algoritmos Evolutivos - Algoritmo Genético</i></b></h1></marquee>

## 1. Import de Bibliotecas e Preparação

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.style as style
%matplotlib inline

## 2. HyperParametros

In [3]:
Qtd_Cromossomos = 6
Loop_Geracoes = 25
bias = 1

## 3. Base de Dados

In [4]:
datasetCancer = pd.read_csv('breastCancer.csv', index_col=0)
datasetCancer.head()

,clump_thickness,size_uniformity,shape_uniformity,marginal_adhesion,epithelial_size,bare_nucleoli,bland_chromatin,normal_nucleoli,mitoses,class
id,,,,,,,,,,
1000025,5,1,1,1,2,1,3,1,1,2
1002945,5,4,4,5,7,10,3,2,1,2
1015425,3,1,1,1,2,2,3,1,1,2
1016277,6,8,8,1,3,4,3,7,1,2
1017023,4,1,1,3,2,1,3,1,1,2


In [5]:
datasetCancer.describe()

,clump_thickness,size_uniformity,shape_uniformity,marginal_adhesion,epithelial_size,bare_nucleoli,bland_chromatin,normal_nucleoli,mitoses,class
count,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,4.417740,3.134478,3.207439,2.806867,3.216023,3.463519,3.437768,2.866953,1.589413,2.689557
std,2.815741,3.051459,2.971913,2.855379,2.214300,3.640708,2.438364,3.053634,1.715078,0.951273
min,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,2.000000
25%,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000
50%,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,3.000000,1.000000,1.000000,2.000000
75%,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,5.000000,4.000000,1.000000,4.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


## 4. DataSet e Preparação dos Dados

- Padronização entre 0 e 1

$$\Huge Normalização(X) = \frac{X - X_{min}}{X_{max} - X_{min}}$$

In [6]:
def normalizacao(Dados):
    return (Dados - Dados.min()) / (Dados.max() - Dados.min())

In [7]:
datasetCancer = datasetCancer.apply(normalizacao)
datasetCancer["class"] = datasetCancer["class"].astype(int)
datasetCancer.describe()

,clump_thickness,size_uniformity,shape_uniformity,marginal_adhesion,epithelial_size,bare_nucleoli,bland_chromatin,normal_nucleoli,mitoses,class
count,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,0.379749,0.237164,0.245271,0.200763,0.246225,0.346352,0.270863,0.207439,0.065490,0.344778
std,0.312860,0.339051,0.330213,0.317264,0.246033,0.364071,0.270929,0.339293,0.190564,0.475636
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.111111,0.000000,0.000000,0.000000,0.111111,0.100000,0.111111,0.000000,0.000000,0.000000
50%,0.333333,0.000000,0.000000,0.000000,0.111111,0.100000,0.222222,0.000000,0.000000,0.000000
75%,0.555556,0.444444,0.444444,0.333333,0.333333,0.500000,0.444444,0.333333,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
datasetCancer["class"].value_counts()

0    458
1    241
Name: class, dtype: int64

In [9]:
Matrix_X = datasetCancer.iloc[0:, 0:-1]
if bias == 1:
    Matrix_X[Matrix_X.shape[1]] = 1
y = datasetCancer.iloc[0:, -1]
Colunas = len(Matrix_X.columns)
Matrix_X.head()

,clump_thickness,size_uniformity,shape_uniformity,marginal_adhesion,epithelial_size,bare_nucleoli,bland_chromatin,normal_nucleoli,mitoses,9
id,,,,,,,,,,
1000025,0.444444,0.000000,0.000000,0.000000,0.111111,0.1,0.222222,0.000000,0.0,1
1002945,0.444444,0.333333,0.333333,0.444444,0.666667,1.0,0.222222,0.111111,0.0,1
1015425,0.222222,0.000000,0.000000,0.000000,0.111111,0.2,0.222222,0.000000,0.0,1
1016277,0.555556,0.777778,0.777778,0.000000,0.222222,0.4,0.222222,0.666667,0.0,1
1017023,0.333333,0.000000,0.000000,0.222222,0.111111,0.1,0.222222,0.000000,0.0,1


## 5. Banco de Cromossomos

- Chutar os pesos

In [10]:
def criar_banco_cromossomos(numero_cromossomos, numero_genes):
    banco = np.zeros([numero_cromossomos, numero_genes])
    for i in range(numero_cromossomos):
        sorteio = np.random.uniform(low=-1., high=1.0, size=(numero_genes,))
        banco[i] = sorteio
    return banco

In [11]:
cromossomos = criar_banco_cromossomos(Qtd_Cromossomos, Colunas)
pd.DataFrame(cromossomos)

,0,1,2,3,4,5,6,7,8,9
0,0.940558,0.923122,0.771965,0.387345,-0.316045,-0.125015,-0.041795,-0.682291,0.123634,-0.666235
1,-0.246546,-0.130806,0.268679,-0.469946,-0.615373,-0.270576,-0.448774,-0.251949,-0.204419,-0.353494
2,-0.655494,0.533721,-0.618362,-0.726855,-0.170662,0.220544,-0.105231,0.920929,-0.858696,-0.281482
3,-0.826571,0.081961,0.498481,0.544155,-0.307295,0.360032,0.800133,-0.852324,-0.129760,0.723942
4,-0.040275,0.295165,0.717149,0.458938,0.505956,0.172229,-0.447226,-0.984977,0.481750,-0.122961
5,-0.094515,-0.469985,-0.573221,0.597640,-0.037236,0.675087,0.858474,-0.662618,-0.497207,0.404390


## 6. Calcula Sistema

In [12]:
def calcula_sistema(X, pesos):
    X = np.array(X)
    pesos = np.array(pesos)
    return (X * pesos).sum(axis=1)

In [13]:
pd.DataFrame(calcula_sistema(Matrix_X, cromossomos[0]))

,0
0,-0.305115
1,0.068163
2,-0.526629
3,0.590311
4,-0.323545
5,1.233942
6,-0.835654
7,-0.532860
8,-0.554398
9,-0.302408


## 7. Acurácia

- 0 = "Não Cancer 4"
- 1 = "Cancer 4"

$$\Huge Fitness = \frac{VP * VN}{TP * TN}$$

- VP = Verdadeiro Positivo
- VN = Verdadeiro Negativo
- TP = Total Positivo
- TN = Total Negativos

https://www.codecogs.com/latex/eqneditor.php?lang=pt-br

In [14]:
def acuracia_fitnes(y_tes, y_pred):
    VP = 0
    VN = 0
    TP = 0
    TN = 0
    y_tes = np.array(y_tes)
    y_pred = np.array(y_pred)
    
    for i in range(len(y_tes)):
        if(y_tes[i] > 0.5):
            TP = TP + 1
            if(y_pred[i] > 0.5):
                VP = VP + 1
        if(y_tes[i] < 0.5):
            TN = TN + 1
            if(y_pred[i] < 0.5):
                VN = VN + 1
            
    return (VP * VN) / (TP * TN)

In [15]:
acuracia_fitnes(y, calcula_sistema(Matrix_X, cromossomos[0]))

0.5647411621881172

## 8.  Roda do Acaso
- Pode ser usado com PROCV no Excel

In [16]:
def definir_peso(fitness_cromossomos):
    peso_fit = np.zeros(fitness_cromossomos.shape[0])
    for i in range(fitness_cromossomos.shape[0]):
        pesos_fit[i] = fitness_cromossomos[i] / fitness_cromossomos.sum()
    return pesos_fit

In [17]:
def roda_do_acaso(pesos_fit):
    a = np.cumsum(pesos_fit)
    b = np.random.rand()
    for i in range(len(pesos_fit)):
        if b <= a[i]:
            return i

## 9. CrossOver

In [18]:
def cross_over(mae, pai):
    cruzamento = np.random.randint(1, len(mae)-1, size = 1)[0]
    x = mae[:cruzamento]
    y = pai[cruzamento:]
    return np.concatenate((x, y))

## 10. Multação

In [19]:
def x_men(filho):
    gene = np.random.randint(0, len(filho)-1, size = 1)[0]
    sorteio = np.random.uniform(low=-1., high=1., size=(1,))
    filho[gene] = sorteio
    return filho

## 11. Retirar Fitness Baixos

In [20]:
def menor_fit(fitness):
    a = np.where(np.array(fitness) == np.array(fitness).min())[0][0]
    return a

## 12. Algoritmo Genético - Modelo Completo

In [21]:
cromossomos = criar_banco_cromossomos(Qtd_Cromossomos, Colunas)
pd.DataFrame(cromossomos)

,0,1,2,3,4,5,6,7,8,9
0,0.224949,-0.637014,-0.754110,0.229068,-0.609380,0.259197,-0.396951,-0.736991,-0.869154,0.302116
1,-0.487744,-0.870267,0.303088,0.953963,-0.657100,-0.268815,0.475971,0.736694,-0.179556,-0.091694
2,0.884653,-0.156952,0.997469,-0.507347,0.434727,-0.167764,-0.428820,-0.472920,-0.905593,-0.489953
3,0.501781,0.781142,-0.342977,-0.214457,0.343477,0.318371,-0.059732,0.220306,0.217431,-0.088603
4,-0.950311,-0.735095,-0.265132,0.370641,-0.994213,0.430790,-0.066162,0.675800,0.676888,0.782840
5,0.244876,0.176386,-0.740850,-0.048763,-0.902331,0.599963,0.611007,0.275325,-0.488455,-0.419683


In [22]:
for i in range(Loop_Geracoes): # Gerações
    fits = np.zeros(cromossomos.shape[0])
    pesos_fit = np.zeros(cromossomos.shape[0])
    
    # Calculando os Fitness
    for j in range(cromossomos.shape[0]): # Individuo Cromossomo
        fits[j] = acuracia_fitnes(y, calcula_sistema(Matrix_X, cromossomos[j]))
    
    # Roda do Acaso
    pesos_fit = definir_peso(fits)
    mae = roda_do_acaso(pesos_fit)
    pai = roda_do_acaso(pesos_fit)
    while mae == pai:
        pai = roda_do_acaso(pesos_fit)
    
        
    # CrosOver
    filho = cross_over(cromossomos[mae], cromossomos[pai])
    indice_menor = menor_fit(fits)
    if (acuracia_fitnes(y, calcula_sistema(Matrix_X, filho)) >
        acuracia_fitnes(y, calcula_sistema(Matrix_X, cromossomos[indice_menor]))):
        cromossomos[indice_menor] = filho
    
    # Mutação no Filho - Irmão Gemeo
    filho_mutante = x_men(filho)
    indice_menor = menor_fit(fits)
    if (acuracia_fitnes(y, calcula_sistema(Matrix_X, filho_mutante)) >
        acuracia_fitnes(y, calcula_sistema(Matrix_X, cromossomos[indice_menor]))):
        cromossomos[indice_menor] = filho
    
Resultado = pd.DataFrame(cromossomos)
Resultado["Fitness"] = fits
Resultado.sort_values(by=['Fitness'], ascending=False)

,0,1,2,3,4,5,6,7,8,9,Fitness
0,0.501781,0.781142,0.499127,-0.214457,0.343477,0.318371,-0.059732,0.220306,0.217431,-0.088603,0.94623
1,0.501781,0.781142,0.499127,-0.214457,0.343477,0.318371,-0.059732,0.220306,0.217431,-0.088603,0.94623
2,0.501781,0.781142,0.499127,-0.214457,0.343477,0.318371,-0.059732,0.220306,0.217431,-0.088603,0.94623
3,0.501781,0.781142,0.499127,-0.214457,0.343477,0.318371,-0.059732,0.220306,0.217431,-0.088603,0.94623
4,0.501781,0.781142,0.499127,-0.214457,0.343477,0.318371,-0.059732,0.220306,0.217431,-0.088603,0.94623
5,0.501781,0.781142,0.499127,-0.214457,0.343477,0.318371,-0.059732,0.220306,0.217431,-0.088603,0.94623
